## Environment setup

For optimal training speed, ensure to enable the GPU of Colab!
Go to `Runtime>Change runtime type`, then select `GPU` in `Hardware accelerator` and click on `Save`.

In [1]:
!nvidia-smi

Mon Oct 27 15:00:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:              6
    Model:                   85
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                3
    BogoMIPS:                4000.44
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

## Installation

In this tutorial, you will need the entire project codebase. So first, we clone the project's GitHub repository and install from source.


In [3]:
!git clone https://github.com/frgfm/torch-cam.git
!uv pip install -e 'torch-cam/.[scripts]'

fatal: destination path 'torch-cam' already exists and is not an empty directory.
Using Python 3.12.12 environment at: /usr
Resolved 43 packages in 123ms
Prepared 1 package in 7ms
Uninstalled 1 package in 0.40ms
Installed 1 package in 1ms
 ~ torchcam==0.4.1.dev0 (from file:///content/torch-cam)


Now we'll download the files we need for the script to work

In [5]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xzf imagenette2-320.tgz
!rm imagenette2-320.tgz

--2025-10-27 15:00:33--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.103.152, 52.216.208.64, 54.231.139.24, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.103.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341663724 (326M) [application/x-tar]
Saving to: ‘imagenette2-320.tgz’

imagenette2-320.tgz 100%[===================>] 325.83M  52.1MB/s    in 6.4s    

2025-10-27 15:00:39 (51.0 MB/s) - ‘imagenette2-320.tgz’ saved [341663724/341663724]



## Running the benchmark

The benchmark script has many parameters for you to experiment on, you can use the help command to inspect them!

In [6]:
!python torch-cam/scripts/eval_perf.py --help

usage: eval_perf.py [-h] [--arch ARCH] [--target TARGET] [--size SIZE]
                    [-b BATCH_SIZE] [--device DEVICE] [-j WORKERS]
                    data_path method

CAM method performance evaluation

positional arguments:
  data_path             path to dataset folder
  method                CAM method to use

options:
  -h, --help            show this help message and exit
  --arch ARCH           Name of the torchvision architecture (default:
                        mobilenet_v3_large)
  --target TARGET       Target layer name (default: None)
  --size SIZE           The image input size (default: 224)
  -b BATCH_SIZE, --batch-size BATCH_SIZE
                        batch size (default: 32)
  --device DEVICE       Default device to perform computation on (default:
                        None)
  -j WORKERS, --workers WORKERS
                        number of data loading workers (default: 2)


Now let's run the benchmark on the downloaded dataset for a given model architecture

In [7]:
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAM --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAMpp --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ SmoothGradCAMpp --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ XGradCAM --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ LayerCAM --arch mobilenet_v3_large --target features

# Other CAM methods run out of VRAM

GradCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.78%, Increase in Confidence 34.83%, Skipped 0 samples
GradCAMpp w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 31.82%, Increase in Confidence 25.35%, Skipped 0 samples
SmoothGradCAMpp w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.90%, Increase in Confidence 26.78%, Skipped 0 samples
XGradCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.78%, Increase in Confidence 34.83%, Skipped 0 samples
LayerCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 25.26%, Increase in Confidence 28.82%, Skipped 0 samples


In [8]:
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAM --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAMpp --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ SmoothGradCAMpp --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ XGradCAM --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ LayerCAM --arch resnet18 --target layer4

# Other CAM methods run out of VRAM

GradCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.86%, Increase in Confidence 22.50%, Skipped 0 samples
GradCAMpp w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 52.71%, Increase in Confidence 19.62%, Skipped 0 samples
SmoothGradCAMpp w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 20.96%, Increase in Confidence 24.82%, Skipped 0 samples
XGradCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.86%, Increase in Confidence 22.50%, Skipped 0 samples
LayerCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 18.05%, Increase in Confidence 28.94%, Skipped 0 samples
